##### Copyright 2018 The TensorFlow Authors.

# Flower Classification using tf.keras

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Loading

In [0]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, 
                                   fname="flower_photos.tgz", 
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

The  dataset we downloaded contains images of 5 types of flowers:

1. Rose
2. Daisy
3. Dandelion
4. Sunflowers
5. Tulips

So, let's create the labels for these 5 classes: 

In [0]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

In [0]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

# Data Augmentation

In [0]:
batch_size =100
IMG_SHAPE = 150

In [0]:
image_gen_train = ImageDataGenerator(rescale=1./255, rotation_range=30, zoom_range=[0.3,0.3], horizontal_flip=True)

train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size, 
                                                     directory=train_dir, 
                                                     shuffle=True, 
                                                     target_size=(IMG_SHAPE,IMG_SHAPE))

### TODO: Create a Data Generator for the Validation Set

Generally, we only apply data augmentation to our training examples. So, in the cell below, use ImageDataGenerator to create a transformation that only rescales the images by 255. Then use the `.flow_from_directory` method to apply the above transformation to the images in our validation set. Make sure you indicate the batch size, the path to the directory of the validation images, the target size for the images, and to set the class mode to `sparse`. Remember that it is not necessary to shuffle the images in the validation set. 

In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size, 
                                                     directory=train_dir, 
                                                     shuffle=True, 
                                                     target_size=(IMG_SHAPE,IMG_SHAPE))

# TODO: Create the CNN

In the cell below, create a convolutional neural network that consists of 3 convolution blocks. Each convolutional block contains a `Conv2D` layer followed by a max pool layer.  The first convolutional block should have 16 filters, the second one should have 32 filters, and the third one should have 64 filters. All convolutional filters should be 3 x 3. All max pool layers should have a `pool_size` of `(2, 2)` . 

After the 3 convolutional blocks you should have a flatten layer followed by a fully connected layer with 512 units. The CNN should output class probabilities based on 5 classes which is done by the **softmax** activation function. All other layers should use a **relu** activation function. 

In [0]:
model = model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(64, (3,3), input_shape=(150, 150, 3)),
                                            tf.keras.layers.BatchNormalization(),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D(2, 2),
                                            tf.keras.layers.Dropout(0.2),
                                            tf.keras.layers.Conv2D(32, (3,3), input_shape=(150, 150, 3)),
                                            tf.keras.layers.BatchNormalization(),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D(2, 2),
                                            tf.keras.layers.Dropout(0.2),
                                            tf.keras.layers.Conv2D(16, (3,3), input_shape=(150, 150, 3)),
                                            tf.keras.layers.BatchNormalization(),
                                            tf.keras.layers.LeakyReLU(),
                                            tf.keras.layers.MaxPooling2D(2, 2),
                                            tf.keras.layers.Dropout(0.2),
                                       
                                            tf.keras.layers.Flatten(),
                                            tf.keras.layers.Dense(512, activation='relu'),
                                            tf.keras.layers.Dropout(0.5),
                                            tf.keras.layers.Dense(256, activation='relu'),
                                            tf.keras.layers.Dropout(0.5),
                                            tf.keras.layers.Dense(5, activation='sigmoid')
                                            ])

# TODO: Compile the Model

In the cell below, compile your model using the ADAM optimizer, the sparse cross entropy function as a loss function. We would also like to look at training and validation accuracy on each epoch as we train our network, so make sure you also pass the metrics argument.

In [0]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer = optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
total_train =sum([len(os.listdir(os.path.join(train_dir, x))) for x in os.listdir(train_dir)])
total_val = sum([len(os.listdir(os.path.join(val_dir, x))) for x in os.listdir(val_dir)])

# TODO: Train the Model

In the cell below, train your model using the  **fit_generator** function instead of the usual **fit** function. We have to use the `fit_generator` function because we are using the **ImageDataGenerator** class to generate batches of training and validation data for our model. Train the model for 80 epochs and make sure you use the proper parameters in the `fit_generator` function . 

In [0]:
epochs = 10

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(batch_size))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(batch_size))),
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss')]
    )

# TODO: Plot Training and Validation Graphs.

In the cell below, plot the training and validation accuracy/loss graphs.

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(7)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

